In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import re
import string

In [109]:
def preprocessing(pwd):
    # import data
    melon = pd.read_csv(pwd, encoding='cp949', keep_default_na=False, na_values="").iloc[:,1:]

    # drop duplicates
    melon.drop_duplicates(inplace=True)

    # convert data
    melon.flac.fillna('No_Flac', inplace=True)
    melon.like = melon.like.str.replace("[^0-9]", "")
    melon.reply = melon.reply.str.replace("[^0-9]", "")

    # drop unnecessary features
    melon.drop(["lyricist", "composer", "arranger"], axis=1, inplace=True)

    # delete rows with null
    temp = melon.isnull().any(axis=1)
    melon = melon.loc[-temp,:]

    # delete rows with no like
    temp = melon.like == ""
    melon = melon.loc[-temp,:]

    # convert type of features
    melon[["like", "reply"]] = melon[["like", "reply"]].astype("int64")

    # delete unnecessary genres
    temp = melon.genre.value_counts() > 10
    genre_list = melon.genre.value_counts()[temp].index

    temp = ["Korean Traditional", "Pop", "Vocal/Choral", "가톨릭음악", "국내CCM", "기타", "동요", "워십", "창작동요"]
    genre_list = [item for item in genre_list if item not in temp]

    temp = melon.genre.isin(genre_list)
    melon = melon.loc[temp,:]

    # compress genre
    melon.genre = ["Animation / Game" if item in ["Animation", "Game"] else   
                   "Crossover / Musical" if item in ["Crossover", "Musical"] else
                   "Drama / Korean Movie" if item in ["Drama", "Korean Movie"] else
                   "Blues / Jazz / New Age" if item in ["Blues", "Jazz", "New Age"] else
                   "Electronica / Rock" if item in ["Electronica", "Rock", "Electronica,Rock"] else
                   item for item in melon.genre]
    return melon

if __name__ == "__main__":
    # directory of melon.csv
    pwd = "./bind_data_melon/melon.csv"
    melon = preprocessing(pwd)

In [61]:
# select columns with null
melon.isnull().any(axis=0)

title     False
artist    False
album     False
date      False
genre     False
flac      False
like      False
reply     False
lyric     False
dtype: bool